# **SETUP**

## Libs && Envs

In [1]:
import os
import glob
from pathlib import Path
import subprocess
import sys
import shutil

import itertools
import random
import pandas as pd
import math
import csv
import re
from collections import OrderedDict, defaultdict
from typing import List, Dict, Tuple
from collections import Counter

import traci
import sumolib
import xml.etree.ElementTree as ET
from pyproj import Transformer, CRS
import networkx as nx
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
from shapely.errors import TopologicalError

from helpers import *

# export LD_LIBRARY_PATH=~/Libs/libnsl
# export SUMO_HOME=~/Envs/sumo-env/lib/python3.10/site-packages/sumo
os.environ["LD_LIBRARY_PATH"] = os.path.expanduser("~/Libs/libnsl")
os.environ["SUMO_HOME"] = os.path.expanduser("~/Envs/sumo-env/lib/python3.10/site-packages/sumo")


## Configs

In [ ]:
# Fixed PATHs
NET_XML = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/newtest-osm.net.xml")
POLY_XML = "/home/hoai-linh.dao/Works/EVCS/AMP-Metropole/Task-1-Completion/results/p0/newtest-poly/bassin-based.poly.xml"
ORIG_VTYPES_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/integrated-dist.add.xml"
GROUPED_POLY_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/group-based.poly.xml"
FLOW_CSV = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/flow.csv"
MAIN_FLOW_CSV = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/main-flow.csv"

SUMO_TOOLS_DIR = Path("/home/hoai-linh.dao/Envs/sumo-env/lib/python3.10/site-packages/sumo/tools")
REROUTING_PY = SUMO_TOOLS_DIR / "generateContinuousRerouters.py"
NETCHECK_PY = SUMO_TOOLS_DIR / "net/netcheck.py"
RANDOMTRIPS_PY = SUMO_TOOLS_DIR / "randomTrips.py"
FINDALLROUTES_PY = SUMO_TOOLS_DIR / "findAllRoutes.py"
PLOTXMLATTRIBUTES_PY = SUMO_TOOLS_DIR / "visualization/plotXMLAttributes.py"
PLOTTRAJECTORIES_PY = SUMO_TOOLS_DIR / "plot_trajectories.py"
PLOTNETDUMP_PY = SUMO_TOOLS_DIR / "visualization/plot_net_dump.py"
PLOTNETSPEED_PY = SUMO_TOOLS_DIR / "visualization/plot_net_speed.py"
PLOTNETTRAFFICLIGHTS_PY = SUMO_TOOLS_DIR / "visualization/plot_net_trafficLights.py"
PLOTSUMMARY_PY = SUMO_TOOLS_DIR / "visualization/plot_summary.py"
PLOTTRIPINFODISTRIBUTIONS_PY = SUMO_TOOLS_DIR / "visualization/plot_tripinfo_distributions.py"
PLOTCSVTIMELINE_PY = SUMO_TOOLS_DIR / "visualization/plot_csv_timeline.py"
PLOTCSVPIE_PY = SUMO_TOOLS_DIR / "visualization/plot_csv_pie.py"
PLOTCSVBARS_PY = SUMO_TOOLS_DIR / "visualization/plot_csv_bars.py"
MACROUTPUT_PY = SUMO_TOOLS_DIR / "visualization/marcoOutput.py"
ROUTESTATS_PY = SUMO_TOOLS_DIR / "route/routeStats.py"
ROUTECHECK_PY = SUMO_TOOLS_DIR / "route/routecheck.py"

# Dynamic DIRs
SIMULATION_DIR = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/simulation-debug-1%")

ODS_DIR = SIMULATION_DIR / "ods"
TRIPS_DIR = SIMULATION_DIR / "trips"
OUTPUTS_DIR = SIMULATION_DIR / "outputs"
LOGS_DIR = SIMULATION_DIR / "logs"
VISUALIZATIONS_DIR = SIMULATION_DIR / "visualizations"

SIMULATION_DIR.mkdir(parents=True, exist_ok=True)
for path in [ODS_DIR, TRIPS_DIR, OUTPUTS_DIR, LOGS_DIR, VISUALIZATIONS_DIR]:
    path.mkdir(parents=True, exist_ok=True)

# Dynamic PATHs
TAZ_XML = SIMULATION_DIR / "taz.add.xml"
VTYPES_DIST_XML = SIMULATION_DIR / "vtypes-dist.add.xml"
ALL_TRIPS_XML = SIMULATION_DIR / "trips.xml"
ROUTE_XML = SIMULATION_DIR / "route.xml"
ROUTE_ALT_XML = SIMULATION_DIR / "route.alt.xml"
REROUTER_XML = SIMULATION_DIR / "rerouter.add.xml"
SUMOCFG_XML = SIMULATION_DIR / "run.sumocfg"

DUAROUTER_LOG = LOGS_DIR / "duarouter.log"
SIMULATION_LOG = LOGS_DIR / "sumo_run.log"
REROUTING_LOG = LOGS_DIR / "rerouting.log"

# Outputs Paths
COLLISIONS_XML = OUTPUTS_DIR / "collisions.xml"
BATTERY_XML = OUTPUTS_DIR / "battery.xml"
LANECHANGES_XML = OUTPUTS_DIR / "laneChanges.xml"
STATISTICS_XML = OUTPUTS_DIR / "statistics.xml"
TRACE_XML = OUTPUTS_DIR / "sumoTrace.xml"
SUMMARY_XML = OUTPUTS_DIR / "summary.xml"
TRIPINFO_XML = OUTPUTS_DIR / "tripinfo.xml"
VEHROUTES_XML = OUTPUTS_DIR / "vehRoutes.xml"
NETSTATE_XML = OUTPUTS_DIR / "netstate.xml"
LOG_TXT = OUTPUTS_DIR / "log.txt"

# Visualization Paths
PLOT_1_PNG = VISUALIZATIONS_DIR / "plot_1.png"
PLOT_2_PNG = VISUALIZATIONS_DIR / "plot_2.png"
PLOT_3_PNG = VISUALIZATIONS_DIR / "plot_3.png"
PLOT_4_PNG = VISUALIZATIONS_DIR / "plot_4.png"
PLOT_5_PNG = VISUALIZATIONS_DIR / "plot_5.png"
PLOT_6_PNG = VISUALIZATIONS_DIR / "plot_6.png"
PLOT_7_PNG = VISUALIZATIONS_DIR / "plot_7.png"
PLOT_8_PNG = VISUALIZATIONS_DIR / "plot_8.png"

# Net-Repairment Task
NET_REPAIRMENT_DIR = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-repairment")
CLEANED_NET_XML_1 = NET_REPAIRMENT_DIR /  f"cleaned_1_{NET_XML.name}"
CLEANED_NET_XML_2 = NET_REPAIRMENT_DIR /  f"cleaned_2_{NET_XML.name}"

KEEP_EDGES_TXT_1 = NET_REPAIRMENT_DIR / "keep-edges_1.txt"
KEEP_EDGES_TXT_2 = NET_REPAIRMENT_DIR / "keep-edges_2.txt"
COMPONENTS_NW_TXT_1 = NET_REPAIRMENT_DIR / "components_nw_1.txt"
COMPONENTS_NW_TXT_2 = NET_REPAIRMENT_DIR / "components_nw_2.txt"

NET_REPAIRMENT_LOGS_DIR = NET_REPAIRMENT_DIR / "logs"
NETCHECK_LOG_1 = NET_REPAIRMENT_LOGS_DIR / "netcheck_1.log"
NETCHECK_LOG_2 = NET_REPAIRMENT_LOGS_DIR / "netcheck_2.log"
NETCHECK_LOG_3 = NET_REPAIRMENT_LOGS_DIR / "netcheck_3.log"
NETCHECK_LOG_4 = NET_REPAIRMENT_LOGS_DIR / "netcheck_4.log"

NETCONVERT_LOG_1 = NET_REPAIRMENT_LOGS_DIR / "netconvert_1.log"
NETCONVERT_LOG_2 = NET_REPAIRMENT_LOGS_DIR / "netconvert_2.log"
NETCONVERT_LOG_3 = NET_REPAIRMENT_LOGS_DIR / "netconvert_3.log"



In [ ]:
TAZ_IDS = {
    'marseille': '1',
    'aix-en-provence': '2',
    'est-etang-de-berre': '3',
    'nord-ouest': '4',
    'ouest-etang-de-berre': '5',
    'sud-est': '6',
    'hors_amp': '99'
}

BORDER_RATIO = 0.40
REAL_ORIGIN   = 'marseille'

CAR_PREFIX = "carDist"              
EV_BRANDS = ["Renault", "Tesla", "Citroen", "Peugeot", "Dacia", "Volkswagen", "BMW", "Fiat", "KIA"]

EV_RATIO = 0.20

DIST_ID = "vehDist"

# Page 11
INCOMING_RATIO = 178729/(178729 + 174729)
OUTGOING_RATIO = 174729/(178729 + 174729)
INCOMING_RATIO, OUTGOING_RATIO

# Page 14 + Page 15
TRIPS_RATIO_0 = 1 # default
TRIPS_RATIO_1 = 0.40 # Marseille 
TRIPS_RATIO_2 = 0.41 # Marseille Bassin
TRIPS_RATIO_3 = 0.52 # AMP Bassin = CEREMA
TRIPS_RATIO_4 = 0.10 # test
TRIPS_RATIO_5 = 0.01 # fast test

PATH_REPLACEMENTS = {
    'net-file': CLEANED_NET_XML_2,
    'route-files': ROUTE_XML,
    'summary-output': SUMMARY_XML,
    'tripinfo-output': TRIPINFO_XML,
    'fcd-output': TRACE_XML,
    'lanechange-output': LANECHANGES_XML,
    'battery-output': BATTERY_XML,
    'vehroute-output': VEHROUTES_XML,
    'collision-output': COLLISIONS_XML,
    'netstate-dump': NETSTATE_XML,
    'statistic-output': STATISTICS_XML,
    'log': LOG_TXT
}


# **PREPARATION**

## Processing Raw Flow

In [ ]:
df = pd.read_csv(FLOW_CSV)

columns_inter = ['Est_Etang-de-Berre','Aix-en-Provence','Sud-Est','Ouest_Etang-de-Berre','Nord-Ouest','Hors_AMP']
df["Intra"] = df["Total"] - df[columns_inter].sum(axis=1)

df.columns = [col.lower() for col in df.columns]

df.to_csv(MAIN_FLOW_CSV, index=False)


## Repairing Network

In [ ]:
NETCHECK_CMD_1 = [
    "python", NETCHECK_PY,
    NET_XML,
    "--vclass", "passenger",
    "--component-output", COMPONENTS_NW_TXT_1
]

with open(NETCHECK_LOG_1, "w") as f:
    print(f"Running NETCHECK Step 1 ...")
    subprocess.run(NETCHECK_CMD_1, stdout=f, stderr=subprocess.STDOUT, check=True)
    print(f"[DONE] Components Ouput written to {COMPONENTS_NW_TXT_1}\n[LOG] Output logged in {NETCHECK_LOG_1}")

print()
print(f"Running extractMaxComponent ...")
extractMaxComponent(COMPONENTS_NW_TXT_1, KEEP_EDGES_TXT_1)


In [ ]:
NETCONVERT_CMD_1 = [
    "netconvert",
    "--net-file", NET_XML,
    "--keep-edges.input-file", KEEP_EDGES_TXT_1,
    "--geometry.remove",
    "--geometry.remove.min-length", "2",
    "--geometry.max-segment-length", "20",
    "--geometry.min-dist", "0.1",
    "--geometry.max-angle", "150",
    "--geometry.max-angle.fix",
    "--remove-edges.isolated",
    "--junctions.join",
    "--junctions.join-dist", "60",
    "--roundabouts.guess",
    "--ramps.guess",
    "--keep-edges.by-vclass=passenger",
    "--osm.bike-access=false",
    "--osm.sidewalks=false",
    "--crossings.guess=false",
    "--tls.guess",
    "--tls.guess.threshold", "40",
    "--tls.join",
    "--tls.layout", "incoming",
    "--tls.discard-loaded",
    "--ptstop-output", "/dev/null",
    "--ptline-output", "/dev/null",
    "-o", CLEANED_NET_XML_1
]

with open(NETCONVERT_LOG_1, "w") as f:
    print(f"Running NETCONVERT Step 1 ...")
    subprocess.run(NETCONVERT_CMD_1, stdout=f, stderr=subprocess.STDOUT, check=True)
    print(f"[DONE] Cleaned Network written to {CLEANED_NET_XML_1}\n[LOG] Output logged in {NETCONVERT_LOG_1}")


In [ ]:
NETCHECK_CMD_2 = [
    "python", NETCHECK_PY,
    CLEANED_NET_XML_1,
    "--vclass", "passenger",
    "--component-output", COMPONENTS_NW_TXT_2,
    "-t"
]

with open(NETCHECK_LOG_2, "w") as f:
    print(f"Running NETCHECK Step 2 ...")
    subprocess.run(NETCHECK_CMD_2, stdout=f, stderr=subprocess.STDOUT, check=True)
    print(f"[DONE] Output logged in {NETCHECK_LOG_2}")

print()
print(f"Running extractMaxComponent ...")
extractMaxComponent(COMPONENTS_NW_TXT_2, KEEP_EDGES_TXT_2)


In [ ]:
NETCONVERT_CMD_2 = [
    "netconvert",
    "--net-file", CLEANED_NET_XML_1,
    "--keep-edges.input-file", KEEP_EDGES_TXT_2,
    "-o", CLEANED_NET_XML_2
]

with open(NETCONVERT_LOG_2, "w") as f:
    print(f"Running NETCONVERT Step 2 ...")
    subprocess.run(NETCONVERT_CMD_2, stdout=f, stderr=subprocess.STDOUT, check=True)
    print(f"[DONE] Cleaned Network written to {CLEANED_NET_XML_2}\n[LOG] Output logged in {NETCONVERT_LOG_2}")


In [ ]:
NETCHECK_CMD_3 = [
    "python", NETCHECK_PY,
    CLEANED_NET_XML_2,
    "--vclass", "passenger",
    "-t"

]

with open(NETCHECK_LOG_3, "w") as f:
    print(f"Running NETCHECK Step 3 ...")
    subprocess.run(NETCHECK_CMD_3, stdout=f, stderr=subprocess.STDOUT, check=True)
    print(f"[DONE] Output logged in {NETCHECK_LOG_3}")


In [ ]:
NETCONVERT_CMD_3 = [
    "netconvert",
    "--net-file", CLEANED_NET_XML_2,          # đầu vào: mạng sau bước 2
    "--geometry.min-radius.fix", "10",        # làm mượt cua gấp (<10 m)
    "--geometry.max-angle", "95",             # bỏ cảnh báo góc quá nhỏ
    "--junctions.corner-detail", "6",         # bo góc mịn hơn
    "--junctions.join-dist", "40",            # tránh gộp nút quá xa
    "--connections.guess", "true",            # tự tạo connection thiếu
    "--keep-edges.min-speed", "5",            # giữ đường nhỏ ≥ 5 m/s
    "-o", CLEANED_NET_XML_3                   # đầu ra: mạng sạch cuối
]

with open(NETCONVERT_LOG_3, "w") as f:
    print("Running NETCONVERT Step 3 (geometry & connections cleanup)...")
    subprocess.run(NETCONVERT_CMD_3,
                   stdout=f,
                   stderr=subprocess.STDOUT,
                   check=True)
    print(f"[DONE] Cleaned Network written to {CLEANED_NET_XML_3}"
          f"\n[LOG] Output logged in {NETCONVERT_LOG_3}")

## Checking Route

In [ ]:
ROUTESTATS_CMD = [
    "python", ROUTESTATS_PY,
    ROUTE_ALT_XML,
    # "-n", CLEANED_NET_XML_2,
    "-a", "routeLength",
    "--binwidth", "500",
    "--hist-output", "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/hist.dat"
]
subprocess.run(ROUTESTATS_CMD, check=True)


# **MAIN**

## Creating TAZ - Way 1 (Using with --ignore-errors)

In [ ]:
poly_tree = ET.parse(GROUPED_POLY_XML)
poly_root = poly_tree.getroot()

region_polys = defaultdict(list)
for poly in poly_root.findall("poly"):
    region = poly.get("type")
    shape_str = poly.get("shape")
    if region and shape_str:
        polygon = parseShape(shape_str)
        if polygon is not None:
            region_polys[region].append(polygon)

region_geoms = {}
for region, polys in region_polys.items():
    if polys:
        try:
            region_geoms[region] = unary_union(polys)
        except TopologicalError as e:
            print(f"[ERROR] Topology error in bassin {region}: {e}")

print("[CHECK] Bounding boxes (per basin):")
for region, geom in region_geoms.items():
    print(f"  {region}: {geom.bounds}")


In [ ]:
net = sumolib.net.readNet(CLEANED_NET_XML_2)

# store edges per basin; plus list for those outside any basin ("hors_amp")
edges_by_region = defaultdict(list)
edges_hors      = []

for edge in net.getEdges():
    # skip technical source/sink edges created by netconvert
    if edge.getID().endswith("-source") or edge.getID().endswith("-sink"):
        continue

    shape = edge.getShape()
    if not shape:
        continue
    mid_pt = shape[len(shape) // 2]
    pt = Point(mid_pt[0], mid_pt[1])

    assigned = False
    for region, geom in region_geoms.items():
        if geom.contains(pt):
            edges_by_region[region].append(edge)
            assigned = True
            break
    if not assigned:
        edges_hors.append(edge)

# add "hors_amp" (outside) group
edges_by_region["hors_amp"] = edges_hors

In [ ]:
taz_root = ET.Element("tazs")

for region, edges in edges_by_region.items():
    rid    = region.lower()
    taz_id = TAZ_IDS.get(rid)
    if not taz_id:
        print(f"[ERROR] No TAZ ID for {region}, skip")
        continue

    geom = region_geoms.get(region)
    if geom is None:
        B, I = [], edges[:]
    else:
        B = selectBoundaryEdges(edges, geom, threshold_ratio=0.1)
        I = [e for e in edges if e not in B]

    if region == "hors_amp":
        conns = edges[:]
    else:
        total = len(B) + len(I)
        nB    = int(BORDER_RATIO * total)
        nI    = total - nB
        conns = random.sample(B, min(nB, len(B))) + \
                random.sample(I, min(nI, len(I)))

    print(f"[CHECK] Basin {region}: total={len(edges)} | B={len(B)} | I={len(I)} | selected={len(conns)}")

    cent = geom.centroid if geom is not None else Point(0, 0)
    taz  = ET.SubElement(taz_root, "taz", id=str(taz_id), x=f"{cent.x:.2f}", y=f"{cent.y:.2f}")
    for e in sorted(conns, key=lambda _e: _e.getID()):
        ET.SubElement(taz, "tazSource", id=e.getID(), weight="1.0")
        ET.SubElement(taz, "tazSink",   id=e.getID(), weight="1.0")

ET.ElementTree(taz_root).write(TAZ_XML, encoding="utf-8", xml_declaration=True)
print(f"\n[DONE] TAZ file written to {TAZ_XML}")

## Creating TAZ - Way 2 (More Roburst)

In [ ]:
SAMPLE_INNER = 40  
SAMPLE_CROSS = 40  

In [ ]:
# STEP‑1  Build basin geometries
print("Running Step 1 - Reading bassins ...")
region_geoms = defaultdict(list)
for p in ET.parse(GROUPED_POLY_XML).getroot().findall('poly'):
    reg = p.get('type')
    geom = parseShape(p.get('shape', ''))
    if reg and geom:
        region_geoms[reg].append(geom)
for reg, polys in region_geoms.items():
    region_geoms[reg] = unary_union(polys) if len(polys) > 1 else polys[0]
print(f"[DONE] Step 1")


In [ ]:
# STEP‑2  Scan network & assign edges to basins
print("Running Step 2 - Scan network & assign edges to basins ...")
NET = sumolib.net.readNet(CLEANED_NET_XML_2)

edges_by_region = defaultdict(list)
outside = []

for e in NET.getEdges():
    if not isValidEdge(e):
        continue
    mid = Point(e.getShape()[len(e.getShape())//2])
    placed = False
    for reg, geom in region_geoms.items():
        if geom.contains(mid):
            edges_by_region[reg].append(e)
            placed = True
            break
    if not placed:
        outside.append(e)

edges_by_region['hors_amp'] = outside

# ── Inner‑basin connectivity 
print("--- Step 2a - Inner‑basin connectivity filter ...")
for reg, pool in list(edges_by_region.items()):
    kept = filterReachable(pool, NET, SAMPLE_INNER)
    if len(kept) < len(pool):
        print(f"  – {reg}: removed {len(pool)-len(kept)} isolated edges")
    edges_by_region[reg] = kept

# ── Cross‑basin connectivity 
print("--- Step 2b - Cross‑basin connectivity filter ...")
for reg_from, pool_from in list(edges_by_region.items()):
    others = [e for r, p in edges_by_region.items() if r != reg_from for e in p]
    if not pool_from or not others:
        continue
    keep = []
    for e in pool_from:
        tgt_sample = random.sample(others, min(SAMPLE_CROSS, len(others)))
        ok_out = any(reachable(e, t, NET) for t in tgt_sample)
        ok_in  = any(reachable(t, e, NET) for t in tgt_sample)
        if ok_out and ok_in:
            keep.append(e)
    removed = len(pool_from) - len(keep)
    if removed:
        print(f"  – {reg_from}: removed {removed} edges not reachable cross‑basin")
    edges_by_region[reg_from] = keep
print(f"[DONE] Step 2")


In [ ]:
# STEP‑3  Write TAZ
print("Running Step 3 - Write TAZ ...")
root = ET.Element('tazs')
for reg, pool in edges_by_region.items():
    if not pool:
        continue
    tid = TAZ_IDS.get(reg.lower())
    if tid is None:
        print(f"  ! no TAZ id for basin {reg}; skip")
        continue
    geom = region_geoms.get(reg)
    B = selectBoundaryEdges(pool, geom)
    I = [e for e in pool if e not in B]
    nb = int(BORDER_RATIO*len(pool))
    ni = len(pool)-nb
    chosen = random.sample(B, min(nb, len(B))) + random.sample(I, min(ni, len(I)))

    c = geom.centroid if geom else Point(0,0)
    taz = ET.SubElement(root, 'taz', id=str(tid), x=f"{c.x:.2f}", y=f"{c.y:.2f}")
    for e in sorted(chosen, key=lambda x:x.getID()):
        ET.SubElement(taz,'tazSource', id=e.getID(), weight='1.0')
        ET.SubElement(taz,'tazSink',   id=e.getID(), weight='1.0')

ET.ElementTree(root).write(TAZ_XML, encoding='utf-8', xml_declaration=True)
print(f"[DONE] {TAZ_XML} ready.")

## Creating Ods

In [ ]:
matrix_files = generateOds(
    MAIN_FLOW_CSV,
    ODS_DIR,
    TAZ_IDS,
    real_origin="marseille",
    exclude_cols={"total","intra"},
    trips_ratio=TRIPS_RATIO_4,
    scale_in=INCOMING_RATIO,
    scale_out=OUTGOING_RATIO
)


for hour, path in matrix_files:
    size = os.path.getsize(path)
    print(f"[DONE] OD matrix hour {hour}: {path} ({size} bytes)")
    

## Creating Vtypes Distribution (Optional)

In [ ]:
assignProbabilitiesToVtypes(
    vtypes_xml=ORIG_VTYPES_XML,
    dist_id="vehDist",
    ev_brands=EV_BRANDS,
    ev_ratio=0.2,
    output_xml=VTYPES_DIST_XML
)


## Creating Trips from Ods

In [ ]:
print("Call od2trips for all ...")
trips_files = od2tripsForAll(TAZ_XML, TRIPS_DIR, ODS_DIR, DIST_ID)
print()
print("[DONE] Finished 24 Trips based on hours.")

mergeTrips(TRIPS_DIR, ALL_TRIPS_XML)


## Creating Route

In [ ]:
DUAROUTER_ADDS = ",".join([str(VTYPES_DIST_XML), str(TAZ_XML)])
DUAROUTER_CMD = [
    "duarouter",
    "-n", CLEANED_NET_XML_2,            
    "-r", ALL_TRIPS_XML,            
    # "-a", DUAROUTER_ADDS,
    "-a", VTYPES_DIST_XML,
    # "--keep-vtype-distributions",
    # "--with-taz",
    # "--repair",                      
    # "--remove-loops",               
    "--randomize-flows",         
    "-o", ROUTE_XML,    
    "--log", DUAROUTER_LOG,
    "--exit-times",
    "--named-routes",
    "--route-length",
    "--write-costs"
]

print("Running DUAROUTER Step ...")
subprocess.run(DUAROUTER_CMD, check=True)
print(f"[DONE] Routes written in {ROUTE_XML}\n[LOG] Output logged in {DUAROUTER_LOG}")


## Creating ReRouter (Optional)

In [ ]:
REROUTING_CMD = [
    "python", REROUTING_PY,
    "-n", CLEANED_NET_XML_2,
    "-o", REROUTER_XML,
    "--vclass", "passenger",
]

with open(REROUTING_LOG, "w") as f:
    print("Running REROUTING Step ...")
    subprocess.run(REROUTING_CMD, stdout=f, stderr=subprocess.STDOUT, check=True)
    print(f"[DONE] Rerouter file is created in {REROUTER_XML}\n[LOG] Output logged in {REROUTING_LOG}")
    

## Updating Sumo-Config

In [ ]:
updateSumoCfg(
    cfg_path=SUMOCFG_XML,
    output_path=SUMOCFG_XML,
    replacements=PATH_REPLACEMENTS
)
print(f"[DONE] New Sumo-Configuration in {ROUTE_XML} with all right paths.")


## Running

In [ ]:
SIMULATION_CMD = [
    "sumo",             
    "-c", SUMOCFG_XML,
    "--no-step-log",      
    "--duration-log.statistics",
    "--xml-validation", "never"  
]

with open(SIMULATION_LOG, "w") as f:
    print("Running SUMO simulation ...")
    subprocess.run(SIMULATION_CMD, stdout=f, stderr=subprocess.STDOUT, check=True)
    print(f"[DONE] Simulation outputs are created in {SIMULATION_DIR}\n[LOG] Output logged in {SIMULATION_LOG}")


# **VISUALIZATIONS**

In [ ]:
# Departure times versus arrival times
PLOT_CMD_1 = [
    "python", PLOTXMLATTRIBUTES_PY,
    VEHROUTES_XML,                 
    "-x", "depart",      
    "-y", "arrival",             
    "-o", PLOT_1_PNG,
    "--scatterplot"
]

subprocess.run(PLOT_CMD_1, check=True)


In [ ]:
# All trajectories over time 1
PLOT_CMD_2 = [
    "python", PLOTXMLATTRIBUTES_PY,
    TRACE_XML,                 
    "-x", "x",     
    "-y", "y",             
    "-o", PLOT_2_PNG,
    "--scatterplot"
]

subprocess.run(PLOT_CMD_2, check=True)


In [ ]:
# Multiple timelines from summary-output
PLOT_CMD_3 = [
    "python", PLOTXMLATTRIBUTES_PY,
    SUMMARY_XML,
    "-x", "time",
    "-y", "running,halting",
    "-o", PLOT_3_PNG,
    "--legend"
]
subprocess.run(PLOT_CMD_3, check=True)


In [ ]:
# Depart delay over time from TripInfo data
PLOT_CMD_4 = [
    "python", PLOTXMLATTRIBUTES_PY,
    TRIPINFO_XML,
    "-x", "depart",
    "-y", "departDelay",
    "--xlabel", "depart time [s]",
    "--ylabel", "depart delay [s]",
    "--ylim", "0,40",
    "--xticks", "0,1200,200,10",
    "--yticks", "0,40,5,10",
    "--xgrid", "--ygrid",
    "--title", "depart delay over depart time",
    "--titlesize", "16",
    "-o", PLOT_4_PNG
]
subprocess.run(PLOT_CMD_4, check=True)


In [ ]:
QUERIED_VEH_ID = "carDist1"
# Selected trajectories over time
PLOT_CMD_5 = [
    "python", PLOTXMLATTRIBUTES_PY,
    TRACE_XML,
    "-x", "x",
    "-y", "y",
    "-i", "id",
    "--filter-ids", QUERIED_VEH_ID
    "--scatterplot",
    "--legend",
    "-o", PLOT_5_PNG
]
subprocess.run(PLOT_CMD_4, check=True)